In [2]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')

from datetime import timedelta

In [3]:
from config import key_id, secret_key

In [4]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [5]:
#The url at the end is what makes a paper trading bot
api = tradeapi.REST(key_id, secret_key)

In [6]:
account = api.get_account()
account.daytrading_buying_power

'0'

In [26]:
api.get_account()

Account({   'account_blocked': False,
    'account_number': '010320JEWA',
    'buying_power': '497.9078',
    'cash': '-1132.9761',
    'created_at': '2020-03-20T17:46:19.948568Z',
    'currency': 'USD',
    'daytrade_count': 1,
    'daytrading_buying_power': '0',
    'equity': '2030.8839',
    'id': '176ffd14-7e0e-4957-9ad5-98306425fa04',
    'initial_margin': '1781.93',
    'last_equity': '2039.52',
    'last_maintenance_margin': '10.26',
    'long_market_value': '3163.86',
    'maintenance_margin': '949.158',
    'multiplier': '2',
    'pattern_day_trader': False,
    'portfolio_value': '2030.8839',
    'regt_buying_power': '497.9078',
    'short_market_value': '0',
    'shorting_enabled': True,
    'sma': '2039.52',
    'status': 'ACTIVE',
    'trade_suspended_by_user': False,
    'trading_blocked': False,
    'transfers_blocked': False})

In [7]:
startdate= datetime.datetime.today().strftime("%Y-%m-%d")
yesterdate=(datetime.datetime.today()- timedelta(days=1)).strftime("%Y-%m-%d")


In [8]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['mfi'] = ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        dataframe['ultosc'] = ta.ULTOSC(dataframe['high'],dataframe['low'],dataframe['close'])
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=20, price='close')
        dataframe['adx'] = tab.ADX(dataframe)

        # required for graphing

        return dataframe

In [9]:
account = api.get_account()
print(account.status)
account.buying_power

ACTIVE


'5.31'

In [15]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [16]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.01

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)


In [46]:
assetsymb=[]
trade=True
#while trade==True:
for i in range(1):
    currenthour=time.localtime().tm_hour
    currentminute=time.localtime().tm_min
    currenttime=currenthour*100+currentminute
    
    assetlist=api.list_positions()
    activity_list=api.get_activities()
    banlist=[]
    oklist=[]
    for activity in  activity_list:
        try:
            if activity.transaction_time.date() == api.get_clock().timestamp.date():
                banlist.append(activity.symbol)

        except:
            e=0
    for asset in assetlist:
        assetsymb.append(asset.symbol)
        if asset.symbol not in banlist:
            try:
                if float(asset.unrealized_intraday_plpc) > .01:
                    returned_data = api.polygon.historic_agg_v2(asset.symbol, 1, 'minute',_from=startdate, to=startdate).df
                    #populate exponential moving averages
                    data=populateindicators(returned_data)

                    if data['rsi'][-1] > 71:
                        openPosition = api.get_position(asset.symbol)

                        returned = api.submit_order(asset.symbol,int(openPosition.qty),"sell","market","day") # Market order to fully close position
                        print('Sell',returned)
                        
                        
                    elif data['plusdi'][-1] < 55 and data['minusdi'][-1] > 45:
                        openPosition = api.get_position(asset.symbol)

                        returned = api.submit_order(asset.symbol,int(openPosition.qty),"sell","market","day") # Market order to fully close position
                        print('Sell',returned)
            except:
                print(asset.symbol)
                
                
    if currenthour >= 12: 
        trade=False

    #interval number of seconds to wait
    time.sleep(24)


In [47]:
assetsymb

['WMT', 'SLB', 'MRO', 'LIN', 'CLX', 'CAG', 'ABBV']

In [48]:
banlist

['CLX', 'LIN', 'CLX', 'LIN', 'WMT', 'ABBV', 'SLB', 'MRO']

In [22]:
api.get_clock().timestamp.date()

datetime.date(2020, 4, 30)

In [24]:
activity_list[0].transaction_time.date()

datetime.date(2020, 4, 30)

In [ ]:
trade=True
#while trade==True:
for i in range(1):
    currenthour=time.localtime().tm_hour
    currentminute=time.localtime().tm_min
    currenttime=currenthour*100+currentminute
    
    assetlist=api.list_positions()
    activity_list=api.get_activities()
    banlist=[]
    oklist=[]
    for activity in  activity_list:
        try:
            if activity.transaction_time.date() == api.get_clock().timestamp.date():
                banlist.append(activity.symbol)

        except:
            e=0
    for asset in assetlist:
        
        if asset.symbol not in banlist:
            try:
                if float(asset.unrealized_intraday_plpc) > .01:
                    returned_data = api.polygon.historic_agg_v2(asset.symbol, 1, 'minute',_from=startdate, to=startdate).df
                    #populate exponential moving averages
                    data=populateindicators(returned_data)

                    if data['rsi'][-1] > 71:

                        openPosition = api.get_position(asset.symbol)

                        returned = api.submit_order(symbol,int(openPosition.qty),"sell","market","day") # Market order to fully close position
                        print('Sell',returned)
            except:
                print(asset.symbol)
